In [ ]:
import numpy as np
import pyvista as pv
import torch
from numba import jit, prange

In [ ]:
etching = np.load('./bosch_data_1022_timeit/bosch_sf_step_test_Ar.npy')

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:50, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching = np.load('./bosch_data_1012_ratio08_trench_condition5_300wide/bosch_sf_step_12_sf.npy')

In [ ]:
etching = np.load('./bosch_data_1021_ratio01_hole/bosch_sf_step_12_c4f8.npy')

In [ ]:
etching.shape

In [ ]:
cyan = torch.Tensor(etching[:100, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:100, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:100, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching = np.load('./bosch_data_1007_ratio08/bosch_mask.npy')

In [ ]:
pos1e4_data = np.load('./bosch_data_1011_ratio08_trench_condition5_300wide/pos1e4.npy')
etching = np.load('./bosch_data_1011_ratio08_trench_condition5_300wide/bosch_sf_test2.npy')

In [ ]:
i = 180
parcel = np.load('./bosch_data_1011_ratio08_trench_condition5_300wide/parcel4_{}.npy'.format(i))

In [ ]:
pos1e4_data = np.load('./ddi_check/pos1e4.npy')

In [ ]:
vel1e4_data = np.load('./ddi_check/vel1e4.npy')

In [ ]:
etching = np.load('./ddi_check/film.npy')

In [ ]:
from surface_normalize_bosch import surface_normal

center_with_direction=np.array([[50,50,140]])
range3D=np.array([[0, 100, 0, 100, 0, 800]])
InOrOut=[1]
yield_hist=np.array([None])
celllength=1e-5
center = 100
center_with_direction=np.array([[center,center,75]])
range3D=np.array([[0, 300, 0, 300, 0, 300]])
InOrOut=[1]
yield_hist=np.array([None])
maskTop=40
maskBottom=98
maskStep=10
maskCenter=[center,center]

# maskTop=40
# maskBottom=98
# maskStep=10
# maskCenter=[50, 50]
tstep = 1e-5
surface =  surface_normal(center_with_direction, range3D, InOrOut,celllength, tstep,yield_hist,\
                          maskTop, maskBottom, maskStep, maskCenter)

In [ ]:
pos = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1])

i = np.floor((pos[:]) + 0.5).astype(int)
print(i)

j = np.floor((pos[:])).astype(int)
print(j)

In [ ]:
pos1e4_data = np.load('./ddi_check/pos_27434.npy')
repos1e4_data = np.load('./ddi_check/repos_27434.npy')

In [ ]:
pos1e4_data = np.load('./ddi_check/pos_26115.npy')
repos1e4_data = np.load('./ddi_check/repos_26115.npy')

In [ ]:
pos1e4_data[:10, :3]

In [ ]:
celllength=1e-5

In [ ]:
etching = np.load('./ddi_check/film.npy')

In [ ]:
mask = torch.Tensor(etching[:, :, :,2]==10).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5
geom = pv.Box()
# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green',opacity=0.5)
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching = np.load('./ddi_check/film.npy')
mask = torch.Tensor(etching[:, :, :,2]==10).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5
geom = pv.Box()
# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)

pos1e4_data = np.load('./ddi_check/pos_27447.npy')
repos1e4_data = np.load('./ddi_check/repos_27447.npy')

point_cloud = pv.PolyData(pos1e4_data[:, :3]/celllength)
vectors = pos1e4_data[:, 3:6]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

revectors = repos1e4_data[:, 3:6]

point_cloud['revectors'] = revectors
rearrows = point_cloud.glyph(
    orient='revectors',
    scale=10000,
    factor=3,
)
# Display the arrowscyan

pos1e4_data2 = np.load('./ddi_check/pos_27638.npy')
repos1e4_data2 = np.load('./ddi_check/repos_27638.npy')

point_cloud2 = pv.PolyData(pos1e4_data2[:, :3]/celllength)
vectors2 = pos1e4_data2[:, 3:6]

point_cloud2['vectors'] = vectors2
arrows2 = point_cloud2.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

revectors2 = repos1e4_data2[:, 3:6]

point_cloud2['revectors'] = revectors2
rearrows2 = point_cloud2.glyph(
    orient='revectors',
    scale=10000,
    factor=3,
)

plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=10.0, render_points_as_spheres=True)
plotter.add_mesh(arrows, color='lightblue')
plotter.add_mesh(rearrows, color='red')

plotter.add_mesh(point_cloud2, color='black', point_size=10.0, render_points_as_spheres=True)
plotter.add_mesh(arrows2, color='lightblue')
plotter.add_mesh(rearrows2, color='red')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.add_mesh(maskglyphed, color='green',opacity=0.1)
plotter.enable_eye_dome_lighting()
plotter.show_grid()
plotter.show()

In [ ]:
etching = np.load('./ddi_check/film.npy')
mask = torch.Tensor(etching[:, :, :,2]==10).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5
geom = pv.Box()
# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)

pos1e4_data = np.load('./ddi_check/pos_35071.npy')
repos1e4_data = np.load('./ddi_check/repos_35071.npy')

point_cloud = pv.PolyData(pos1e4_data[:, :3]/celllength)
vectors = pos1e4_data[:, 3:6]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

revectors = repos1e4_data[:, 3:6]

point_cloud['revectors'] = revectors
rearrows = point_cloud.glyph(
    orient='revectors',
    scale=10000,
    factor=3,
)
# Display the arrowscyan

pos1e4_data2 = np.load('./ddi_check/pos_35083.npy')
repos1e4_data2 = np.load('./ddi_check/repos_35083.npy')

point_cloud2 = pv.PolyData(pos1e4_data2[:, :3]/celllength)
vectors2 = pos1e4_data2[:, 3:6]

point_cloud2['vectors'] = vectors2
arrows2 = point_cloud2.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

revectors2 = repos1e4_data2[:, 3:6]

point_cloud2['revectors'] = revectors2
rearrows2 = point_cloud2.glyph(
    orient='revectors',
    scale=10000,
    factor=3,
)

plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=10.0, render_points_as_spheres=True)
plotter.add_mesh(arrows, color='lightblue')
plotter.add_mesh(rearrows, color='red')

plotter.add_mesh(point_cloud2, color='black', point_size=10.0, render_points_as_spheres=True)
plotter.add_mesh(arrows2, color='lightblue')
plotter.add_mesh(rearrows2, color='red')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.add_mesh(maskglyphed, color='green',opacity=0.1)
plotter.enable_eye_dome_lighting()
plotter.show_grid()
plotter.show()

In [ ]:
pos1e4_data = np.load('./ddi_check/pos_26486.npy')
repos1e4_data = np.load('./ddi_check/repos_26486.npy')

In [ ]:
point_cloud = pv.PolyData(pos1e4_data[:, 6:9])
vectors = pos1e4_data[:, 3:6]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

revectors = repos1e4_data[:, 3:6]

point_cloud['revectors'] = revectors
rearrows = point_cloud.glyph(
    orient='revectors',
    scale=10000,
    factor=3,
)
# Display the arrowscyan


plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
plotter.add_mesh(rearrows, color='red')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
pos1e4 = pos1e4_data/1e-5
posshape = np.ones_like(etching)
arr_int = pos1e4.astype(int)
print(posshape.shape)
print(arr_int)

In [ ]:
etching[10, 100, 45:]

In [ ]:
arr_int.shape

In [ ]:
arr_int[:,2].min()

In [ ]:
np.sum(etching, axis=-1).shape

In [ ]:
vel1e4_data.shape

In [ ]:
def SpecularReflect(vel, normal):
    return vel - 2*vel@normal*normal

In [ ]:
testVel = np.random.rand(10000, 3)

testVel[:,0] -= 0.5
testVel[:,1] -= 0.5
# testVel[:,2] *= -1

testVelNorm = np.linalg.norm(testVel, axis=-1)
testVelNormlized = np.zeros_like(testVel)
testVelNormlized[:,0] = testVel[:,0]/testVelNorm
testVelNormlized[:,1] = testVel[:,1]/testVelNorm
testVelNormlized[:,2] = testVel[:,2]/testVelNorm

surNorm = np.zeros_like(testVel)
surNorm[:,2] = 1

In [ ]:
reflect = np.zeros_like(testVelNormlized)
for i in range(reflect.shape[0]):
    reflect[i] = SpecularReflect(testVelNormlized[i], surNorm[i])

In [ ]:
np.sum(reflect[:,2] <= 0)

In [ ]:
point_cloud = pv.PolyData(np.zeros_like(reflect))
vectors = reflect

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

# Display the arrowscyan


plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
react_table = np.array([[[0.1, -1, 0, 0], [0.0, 0,  0, 0], [0.0, 0, 0, 0]],
                        [[0.8, -1, 1, 0], [0.0, 0,  0, 0], [0.0, 0, 0, 0]],
                        [[1.0,  0, 0, 0], [1.0, 0, -2, 0], [1.0, 0, 0, 0]]])

# react_table[0, 3, 4] = -2
# etching act on film, depo need output

# react_type
#       Si c4f8 mask
# sf   ([[KD, x, x],
# c4f8   [+,  x, x],
# Ar     [+, KD, +]])

react_type_table = np.array([[2, 0, 0],
                           [1, 0, 0],
                           [4, 3, 1]])


# @jit(nopython=True, parallel=False)
def reaction_yield(parcel, film, theta):
    # print('react parcel', parcel.shape)
    # print('react film', film.shape)
    # print('react theta', theta.shape)
    num_parcels = parcel.shape[0]
    num_reactions = react_table.shape[1]
    choice = np.random.rand(parcel.shape[0], react_table.shape[1])
    reactList = np.ones(parcel.shape[0], dtype=np.int_)*-1
    for i in range(num_parcels):
        for j in range(num_reactions):
            if film[i, j] <= 0:
                choice[i, j] = 1
    depo_parcel = np.zeros(parcel.shape[0])
    for i in prange(parcel.shape[0]):
        acceptList = np.zeros(react_table.shape[1], dtype=np.bool_)
        for j in prange(film.shape[1]):
            react_rate = react_table[int(parcel[i, -1]), j, 0]
            if react_rate > choice[i, j]:
                acceptList[j] = True
        react_choice_indices = np.where(acceptList)[0]
        print('acceptList:{}, react_choice_indices:{}'.format(acceptList, react_choice_indices))
        # print(react_choice_indices)
        if react_choice_indices.size > 0:
            react_choice = np.random.choice(react_choice_indices)
            reactList[i] = react_choice
            react_type = react_type_table[int(parcel[i, -1]), react_choice]
            if react_type == 2: # kdtree Si-SF
                depo_parcel[i] = 2
            elif react_type == 3: # kdtree Ar-c4f8
                depo_parcel[i] = 3
            elif react_type == 1: # +
                depo_parcel[i] = 1
            elif react_type == 4: # Ar - Si
                depo_parcel[i] = 4
            elif react_type == 0:  # no reaction
                depo_parcel[i] = 0
    for i in prange(parcel.shape[0]):
        if depo_parcel[i] == 1:
            film[i, :] += react_table[int(parcel[i, -1]), int(reactList[i]), 1:]
            # print('chemistry')
        # if reactList[i] == -1:
        #     parcel[i,3:6] = SpecularReflect(parcel[i,3:6], theta[i])
            # print('reflection')
            # parcel[i,3:6] = reemission(parcel[i,3:6], theta[i])

    return film, parcel, reactList, depo_parcel

@jit(nopython=True)
def SpecularReflect(vel, normal):
    return vel - 2*vel@normal*normal

In [ ]:
film = np.array([[10,0,0],[10,0,0],[10,0,0],[0,0,10]])

In [ ]:
parcel = np.array([[1,0],[1,0],[1,0],[1,0]])
# film = np.array([[0,0,10],[0,0,10],[0,0,10],[0,0,10]])

theta = np.array([[1,0],[1,0],[1,0],[1,0]])

test = reaction_yield(parcel, film, theta)

In [ ]:
test[2]

In [ ]:
point_cloud = pv.PolyData(arr_int)
vectors = vel1e4_data

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

# Display the arrowscyan


plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
planes = surface.get_pointcloud(np.sum(etching, axis=-1))



point_cloud = pv.PolyData(planes[:, 3:])
vectors = planes[:, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

# Display the arrowscyan


plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
indice_c3 = np.logical_and(etching[:, :, :, 1]>9, etching[:, :, :, 1]<10)
etching[indice_c3, 1] = 0

In [ ]:
print(parcel[:10, 6:9].astype(int))

In [ ]:
pos1e4_data = np.load('./ddi_check/pos1e4.npy')

In [ ]:
pos1e4 = pos1e4_data/1e-5
posshape = np.ones_like(etching)
arr_int = pos1e4.astype(int)
print(posshape.shape)
print(arr_int)

In [ ]:
np.min(arr_int[:, 2])

In [ ]:
np.max(arr_int[:, 0])

In [ ]:
parcel = np.load('./ddi_check/pos1e4.npy')

In [ ]:
etching = np.load('./ddi_check/film.npy')

In [ ]:
etching[10, 103, 10:50]

In [ ]:
surface_film = np.array(etching[:, :, :,0] < 10)
etching[surface_film, 0] = 0

In [ ]:
parcelshape = np.ones_like(etching)
parcel_int = pos1e4.astype(int)

print(parcel_int.shape)
indice_max_1 = parcel_int[:, 0] > 19
print(np.sum(indice_max_1))

indice_max_2 = parcel_int[:, 1] > 199
print(np.sum(indice_max_2))

indice_max_3 = parcel_int[:, 2] > 199
print(np.sum(indice_max_3))

indice_max_out = np.zeros_like(indice_max_1)
indice_max_out[indice_max_1] = 1
indice_max_out[indice_max_2] = 1
indice_max_out[indice_max_3] = 1

pos1e4 = pos1e4_data/1e-5
posshape = np.ones_like(etching)
arr_int = pos1e4.astype(int)
posshape[arr_int[:,0],arr_int[:,1],arr_int[:,2] ,:] = 10

parcelshape = np.ones_like(etching)
parcel_int = pos1e4.astype(int)
parcel_int = parcel_int[~indice_max_out]
parcelshape[parcel_int[:,0],parcel_int[:,1],parcel_int[:,2] ,:] = 10

In [ ]:
parcelshape = np.ones_like(etching)
parcel_int = parcel[1:, 6:9].astype(int)

print(parcel_int.shape)
indice_max_1 = parcel_int[:, 0] > 19
print(np.sum(indice_max_1))

indice_max_2 = parcel_int[:, 1] > 199
print(np.sum(indice_max_2))

indice_max_3 = parcel_int[:, 2] > 199
print(np.sum(indice_max_3))

indice_max_out = np.zeros_like(indice_max_1)
indice_max_out[indice_max_1] = 1
indice_max_out[indice_max_2] = 1
indice_max_out[indice_max_3] = 1

pos1e4 = pos1e4_data/1e-5
posshape = np.ones_like(etching)
arr_int = pos1e4.astype(int)
posshape[arr_int[:,0],arr_int[:,1],arr_int[:,2] ,:] = 10

parcelshape = np.ones_like(etching)
parcel_int = parcel[1:, 6:9].astype(int)
parcel_int = parcel_int[~indice_max_out]
parcelshape[parcel_int[:,0],parcel_int[:,1],parcel_int[:,2] ,:] = 10

In [ ]:
pos1e4 = pos1e4_data/1e-5
posshape = np.ones_like(etching)
arr_int = pos1e4.astype(int)
posshape[arr_int[:,0],arr_int[:,1],arr_int[:,2] ,:] = 10

parcelshape = np.ones_like(etching)
parcel_int = parcel[1:, 6:9].astype(int)
parcel_int = parcel_int[~indice_max_out]
parcelshape[parcel_int[:,0],parcel_int[:,1],parcel_int[:,2] ,:] = 10

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]==10, etching[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(parcelshape[:, :, :,0] == 10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]==10).to_sparse()
gray = gray.indices().numpy().T

# mask = torch.Tensor(np.logical_and(etching[:, :, :,0]!=0, etching[:, :, :,0]<10)).to_sparse()
mask = torch.Tensor(etching[:, :, :,2]==10).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green',opacity=0.5)
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray',opacity=0.5)
p.enable_eye_dome_lighting()
p.show()

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]==10, etching[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(parcelshape[:, :, :,0] == 10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*1
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray',opacity=0.5)
p.enable_eye_dome_lighting()
p.show()

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]==10, etching[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(parcelshape[:, :, :,0] == 10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray',opacity=0.5)
p.enable_eye_dome_lighting()
p.show()

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]==10, etching[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(parcelshape[:, :, :,0] == 10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*1
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray',opacity=0.5)
p.enable_eye_dome_lighting()
p.show()

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]==10, etching[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(posshape[:, :, :,0] == 10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*1
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray',opacity=0.5)
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching = np.load('./bosch_data_1021_ratio01_hole/bosch_sf_step_1_sf.npy')

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:50, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching = np.load('./bosch_data_1021_ratio01_hole/bosch_sf_step_10_sf.npy')

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:100, :, :,1]==10, etching[:100, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:100, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:100, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:100, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]==10, etching[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:100, :, :,1]==10, etching[:100, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:100, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:100, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:100, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching = np.load('./bosch_data_1017_ratio01_hole/bosch_sf_step_21_sf.npy')

In [ ]:
etching = np.load('./bosch_data_1015_ratio01_hole/bosch_sf_step_1_c4f8.npy')

In [ ]:
etching = np.load('./bosch_data_1016_ratio08_hole/bosch_sf_step_20_sf.npy')

In [ ]:
indice_c3 = np.logical_and(etching[:, :, :, 1]>9, etching[:, :, :, 1]<10)
etching[indice_c3, 1] = 10
indice_step2 = etching[:, :, :, 0] != 10
etching[indice_step2, 0] = 0
indice_c2 = etching[:, :, :, 1] != 0
etching[indice_c2, 1] = 10

In [ ]:
etching = np.load('./bosch_data_1017_ratio01_hole/bosch_sf_step_20_sf.npy')

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:50, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching = np.load('./bosch_data_1017_ratio08_hole/bosch_sf_step_13_sf.npy')

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:50, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching = np.load('./bosch_data_1017_ratio08_hole/bosch_sf_step_2_sf.npy')

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:50, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching = np.load('./bosch_data_1017_ratio01_hole/bosch_sf_step_2_sf.npy')

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:50, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
def thickPolymer(film):  # fast scanZ
    filmC = film[:,:,:,1]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,0] == 10 

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(filmC, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & neighbors

    # 将孤立的单元格设为0
    film[condition, 1] = 10

    all_film = np.logical_and(film[:,:,:,1] == 10, film[:,:,:,0] == 10)
    film[all_film, 0] = 0
    return film

In [ ]:
etching = thickPolymer(etching)

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]==10, etching[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green',opacity=0.5)
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]==10, etching[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green',opacity=0.5)
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray',opacity=0.5)
p.enable_eye_dome_lighting()
p.show()

In [ ]:
def removeFloat(film):  # fast scanZ
    filmC = film[:,:,:,0]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,0] != 0

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(filmC, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & ~neighbors

    # 将孤立的单元格设为0
    film[condition, :] = 0

    return film

In [ ]:
etching = removeFloat(etching)

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]==10, etching[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green',opacity=0.5)
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching = np.load('./bosch_data_1012_ratio08_trench_condition5_300wide/bosch_sf_step_10_sf.npy')

In [ ]:
etching = removeFloat(etching)

In [ ]:
def removeFloat(film):  # fast scanZ
    filmC = film[:,:,:,0]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,1] != 0

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(filmC, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & ~neighbors

    # 将孤立的单元格设为0
    film[condition, :] = 0

    return film

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]==10, etching[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching = np.load('./bosch_data_1015_ratio01_hole/bosch_sf_step_5_sf.npy')

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:50, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:50, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
def removeFloat(film):  # fast scanZ
    filmC = film[:,:,:,0]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,1] != 0

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(filmC, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & ~neighbors

    # 将孤立的单元格设为0
    film[condition, :] = 0

    return film

In [ ]:
etching2 = removeFloat(etching)

In [ ]:
cyan = torch.Tensor(etching2[:, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching2[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching2[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching = np.load('./bosch_data_1013_ratio01_hole/bosch_sf_step_2_sf.npy')

In [ ]:
def attachPolymer(film):  # fast scanZ
    filmC = film[:,:,:,0]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,1] == 0 

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(filmC, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & neighbors

    # 将孤立的单元格设为0
    film[condition, 1] = 10

    all_film = np.logical_and(film[:,:,:,1] == 10, film[:,:,:,0] == 10)
    film[all_film, 1] = 0
    all_film2 = np.logical_and(film[:,:,:,1] == 10, film[:,:,:,2] == 10)
    film[all_film2, 1] = 0
    return film

In [ ]:
etching2 = attachPolymer(etching)

In [ ]:
indice_c3 = np.logical_and(etching[:, :, :, 1]>9, etching[:, :, :, 1]<10)
etching[indice_c3, 1] = 10
indice_step2 = etching[:, :, :, 0] != 10
etching[indice_step2, 0] = 0
indice_c2 = etching[:, :, :, 1] != 0
etching[indice_c2, 1] = 10

In [ ]:
etching[50, 50, :20]

In [ ]:
etching2[50, 50, :20]

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching2[:50, :, :,1]==10, etching2[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching2[:50, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching2[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching2[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching3[:50, :, :,1]==10, etching3[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching3[:50, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching3[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching3[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
# cyan = torch.Tensor(etching[:50, :, :,1]!=0).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
pos1e4.shape

In [ ]:
planes = surface.get_pointcloud(np.sum(etching, axis=-1))

helf_plane = int(planes.shape[0]/2)

point_cloud = pv.PolyData(planes[:helf_plane, 3:])
vectors = planes[:helf_plane, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=10000,
    factor=3,
)

# Display the arrowscyan


plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
print(vectors.shape)

In [ ]:
print(vectors[:10])

In [ ]:
print(np.linalg.norm(vectors[:10],axis=-1))

In [ ]:
planes = surface.get_pointcloud(np.sum(etching, axis=-1))
planes[:10, 3:]

In [ ]:
maskWall_indice = np.logical_and(planes[:, 5] > -102, planes[:, 5] < -158)

In [ ]:
maskWall_indice.shape

In [ ]:
test = planes[maskWall_indice, 3:]

print(test.shape)

In [ ]:
planes.shape

In [ ]:
planes = surface.get_pointcloud(np.sum(etching, axis=-1))
maskWall_indice = np.logical_and(planes[:, 5] > 102, planes[:, 5] < 158)
test = planes[maskWall_indice, 3:]

test[:, 0] -= 50
test[:, 1] -= 50

np.sqrt(test[:, 0]*test[:, 0] + test[:, 1]*test[:, 1] )

vector_z = np.sqrt(test[:, 0]*test[:, 0] + test[:, 1]*test[:, 1])/10

print(vector_z)

new_vector = np.array([-test[:, 0], -test[:, 1], -vector_z]).T
new_vector_norm = np.linalg.norm(new_vector, axis=-1)

new_vector[:, 0] = np.divide(new_vector[:, 0], new_vector_norm)
new_vector[:, 1] = np.divide(new_vector[:, 1], new_vector_norm)
new_vector[:, 2] = np.divide(new_vector[:, 2], new_vector_norm)

planes[maskWall_indice, :3] = new_vector
print(test)

In [ ]:
test.shape

In [ ]:
planes.shape

In [ ]:
np.sqrt(test[:, 0]*test[:, 0] + test[:, 1]*test[:, 1] )

vector_z = np.sqrt(test[:, 0]*test[:, 0] + test[:, 1]*test[:, 1])/10

print(vector_z)

new_vector = np.array([-test[:, 0], -test[:, 1], -vector_z]).T
new_vector_norm = np.linalg.norm(new_vector, axis=-1)

new_vector[:, 0] = np.divide(new_vector[:, 0], new_vector_norm)
new_vector[:, 1] = np.divide(new_vector[:, 1], new_vector_norm)
new_vector[:, 2] = np.divide(new_vector[:, 2], new_vector_norm)


In [ ]:
helf_plane = int(planes.shape[0])
point_cloud = pv.PolyData(planes[:helf_plane, 3:])
vectors = planes[:helf_plane, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=100000,
    factor=3,
)

# Display the arrowscyan

plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
planes = surface.get_pointcloud(np.sum(etching, axis=-1))
helf_plane = int(planes.shape[0]/2)
point_cloud = pv.PolyData(planes[:helf_plane, 3:])
vectors = planes[:helf_plane, :3]

point_cloud['vectors'] = vectors
arrows = point_cloud.glyph(
    orient='vectors',
    scale=100000,
    factor=3,
)

# Display the arrowscyan

plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='cyan', point_size=5.0, render_points_as_spheres=True)
# plotter.add_mesh(sphere, show_edges=True, opacity=0.5, color="w")
plotter.add_mesh(arrows, color='lightblue')
# # plotter.add_point_labels([point_cloud.center,], ['Center',],
# #                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:, 50:, :,1]!=0, etching[:, 50:, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, 50:, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:, 50:, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:50, :, :,1]!=0, etching[:50, :, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etching[50, 50, 65:85]

In [ ]:
etching = np.load('./bosch_data_1002_ratio08/bosch_sf_step_5_c4f8.npy')

In [ ]:
depo1 = torch.Tensor(etching[:50, :, :,1]==10).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:, :, :,1]==10).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:50, :, :,1]!=0, etching[:50, :, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:, :, :,1]==10).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etching[:50, :, :,1]!=0, etching[:50, :, :,0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.sum(etching[:, :, :, :], axis=-1)==0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

# mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
# mask = mask.indices().numpy().T

# maskmesh = pv.PolyData(mask)
# maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
# maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.sum(etching[:, :, bottom-250:bottom, :], axis=-1)==0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, bottom-250:bottom,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:, :, bottom-250:bottom,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:, :, bottom-250:bottom, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, bottom-250:bottom,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:, :, bottom-250:bottom,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
# p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:50, :, bottom-250:bottom+10, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, bottom-250:bottom+10,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:50, :, bottom-250:bottom+10,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:50, :, bottom-100:bottom+10, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:50, :, bottom-100:bottom+10,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etching[:50, :, bottom-100:bottom+10,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()